In [48]:
import numpy as np
from matplotlib.pyplot import cm
from matplotlib.colors import ListedColormap
import imageio
from PIL import Image

In [41]:
input_media = imageio.get_reader("../Dataset/Mahnob/data/media_24/53.avi")

In [11]:
input_media.get_meta_data()

{'duration': 105.08,
 'ffmpeg_version': u'3.4.4-0ubuntu0.18.04.1 built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)',
 'fps': 24.0,
 'nframes': 2522,
 'plugin': 'ffmpeg',
 'size': (320, 200),
 'source_size': (320, 200)}

In [12]:
data = np.load("../export/data/gaze_heatmap_53.npz")

In [65]:
heatmaps = data["heatmaps"]
print heatmaps.shape

(2522, 64000)


In [18]:
# normalize over all frames
norm_heatmap = ( heatmaps - heatmaps.min() ) / ( heatmaps.max() - heatmaps.min() )

In [49]:
cmap = cm.jet
my_cmap = cmap(np.arange(cmap.N))

# Set alpha
my_cmap[:,-1] = np.linspace(0, 1, cmap.N)

# Create new colormap
cm_jet = ListedColormap(my_cmap)

In [56]:
i = 500

output_media = imageio.get_writer("../export/video/sm_53.png")

frame = input_media.get_data(i)
image = Image.fromarray(frame).convert("RGBA")

hm =  cm_jet(norm_heatmap[i].reshape(200,320), bytes=True)
hm_image = Image.fromarray(hm)

image.paste(hm_image, (0,0), hm_image)

output_media.append_data(np.array(image))

output_media.close()

## Export entire video

In [57]:
output_media = imageio.get_writer("../export/video/sm_53.avi", fps=24)

for frame, hm in zip(input_media.iter_data(), norm_heatmap):
    
    image = Image.fromarray(frame).convert("RGBA")
    
    hm =  cm_jet(hm.reshape(200,320), bytes=True)
    hm_image = Image.fromarray(hm)

    image.paste(hm_image, (0,0), hm_image)

    output_media.append_data(np.array(image))

output_media.close()

## Summarize video with saliency values > 0.7

In [85]:
output_media_07 = imageio.get_writer("../export/video/sm_53_07.avi", fps=10)

norm_heatmap_07_index = np.where(norm_heatmap > 0.7)

In [86]:
for i in np.unique(norm_heatmap_07_index[0]):
    
    frame = input_media.get_data(i)
    image = Image.fromarray(frame).convert("RGBA")
    
    hm =  cm_jet(norm_heatmap[i].reshape(200,320), bytes=True)
    hm_image = Image.fromarray(hm)

    image.paste(hm_image, (0,0), hm_image)

    output_media_07.append_data(np.array(image))

output_media_07.close()